In [1]:
import numpy as np
import pandas as pd
import os
import sys
import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

from sklearn.pipeline import Pipeline

PATH = "/kaggle/input/tsp-cv"
PATH_TRAIN = os.path.join(PATH, "train.csv")
PATH_TEST = os.path.join(PATH , 'test.csv')

In [2]:
df_train = pd.read_csv(PATH_TRAIN)
df_test = pd.read_csv(PATH_TEST)
display(df_train)
display(df_test)

,id,filename,distance
0,0,0.jpg,83110
1,1,1.jpg,1035
2,2,2.jpg,20756
3,3,3.jpg,13286
4,4,4.jpg,13924
...,...,...,...
16013,16014,16014.jpg,1803
16014,16015,16015.jpg,12492
16015,16016,16016.jpg,1556
16016,16017,16017.jpg,16507


,id,filename
0,16019,16019.jpg
1,16020,16020.jpg
2,16021,16021.jpg
3,16022,16022.jpg
4,16023,16023.jpg
...,...,...
4000,20019,20019.jpg
4001,20020,20020.jpg
4002,20021,20021.jpg
4003,20022,20022.jpg


We want to use early stopping.  To do this, we need a validation set.  We will break the data into 80 percent test data and 20 validation.  

In [3]:
TRAIN_PCT = 0.9
TRAIN_CUT = int(len(df_train) * TRAIN_PCT)

df_train_cut = df_train[0:TRAIN_CUT]
df_validate_cut = df_train[TRAIN_CUT:]

print(f"Training size: {len(df_train_cut)}")
print(f"Validate size: {len(df_validate_cut)}")
display(df_train_cut)
display(df_validate_cut)

Training size: 14416
Validate size: 1602


,id,filename,distance
0,0,0.jpg,83110
1,1,1.jpg,1035
2,2,2.jpg,20756
3,3,3.jpg,13286
4,4,4.jpg,13924
...,...,...,...
14411,14412,14412.jpg,10152
14412,14413,14413.jpg,4605
14413,14414,14414.jpg,42715
14414,14415,14415.jpg,17875


,id,filename,distance
14416,14417,14417.jpg,18399
14417,14418,14418.jpg,14557
14418,14419,14419.jpg,33278
14419,14420,14420.jpg,9838
14420,14421,14421.jpg,5092
...,...,...,...
16013,16014,16014.jpg,1803
16014,16015,16015.jpg,12492
16015,16016,16016.jpg,1556
16016,16017,16017.jpg,16507


Next, we create the generators that will provide the images to the neural network as it is trained.  We normalize the images so that the RGB colors between 0-255 become ratios between 0 and 1.  We also use the **flow_from_dataframe** generator to connect the Pandas dataframe to the actual image files. We see here a straightforward implementation; you might also wish to use some of the image transformations provided by the data generator.

The **HEIGHT** and **WIDTH** constants specify the dimensions that the image will be scaled (or expanded) to. It is probably not a good idea to expand the images.

In [4]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

# WIDTH = 299
# HEIGHT = 299
WIDTH = 224
HEIGHT = 224

training_datagen = ImageDataGenerator(
  rescale = 1./255,
  horizontal_flip=True,
  #vertical_flip=True,
  fill_mode='nearest')

train_generator = training_datagen.flow_from_dataframe(
        dataframe=df_train_cut,
        directory=PATH,
        x_col="filename",
        y_col="distance",
        target_size=(HEIGHT, WIDTH),
        batch_size=32, # Keeping the training batch size small USUALLY increases performance
        class_mode='raw')

validation_datagen = ImageDataGenerator(rescale = 1./255)

val_generator = validation_datagen.flow_from_dataframe(
        dataframe=df_validate_cut,
        directory=PATH,
        x_col="filename",
        y_col="distance",
        target_size=(HEIGHT, WIDTH),
        batch_size=256, # Make the validation batch size as large as you have memory for
        class_mode='raw')


test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = validation_datagen.flow_from_dataframe(
        dataframe=df_test,
        directory=PATH,
        x_col="filename",
    
        batch_size=1,
        shuffle=False,
        target_size=(HEIGHT, WIDTH),
        class_mode=None)

Found 14416 validated image filenames.
Found 1602 validated image filenames.
Found 4005 validated image filenames.


## Transfer Learning

We will now use a ResNet neural network as a basis for our neural network.  We will redefine both the input shape and output of the ResNet model, so we will not transfer the weights.  Since we redefine the input; the weights are of minimal value.  We begin by loading, from Keras, the ResNet50 network.

In [5]:
from tensorflow.keras.applications import Xception,DenseNet121
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Input
#https://www.kaggle.com/code/abnera/transfer-learning-keras-xception-cnn/script
input_tensor = Input(shape=(HEIGHT, WIDTH, 3))

base_model = DenseNet121(
    include_top=False, weights='imagenet', input_tensor=input_tensor,
    input_shape=None)

#base_model.summary()

2022-04-18 20:05:19.233773: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-18 20:05:19.364471: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-18 20:05:19.365414: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-18 20:05:19.367140: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

29097984/29084464 [==============================] - 2s 0us/step


Now we must add a few layers to the end of the neural network so that it becomes a regression model.

In [6]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
#https://vigneshgig.medium.com/xception-neural-network-transfer-learning-and-data-processing-using-ai-c3e7a4ea7bf2
#https://www.kaggle.com/code/abnera/transfer-learning-keras-xception-cnn/script
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(2048,activation='relu')(x) 
x=Dense(2048,activation='relu')(x) 
model=Model(inputs=base_model.input,outputs=Dense(1)(x))
# predictions = Dense(2, activation='softmax')(x)
# model = Model(base_model.input, predictions)

Now we train just like before, the only difference is that we do not define the entire neural network here.

In [7]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow import keras
# Important, calculate a valid step size for the validation dataset
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
base_model.trainable = True
model.compile(loss = 'mean_squared_error', optimizer=keras.optimizers.Adam(1e-3), metrics=[RootMeanSquaredError(name="rmse")])
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=25, verbose=1, mode='auto',
        restore_best_weights=True)

history = model.fit(train_generator, epochs=200, steps_per_epoch=STEP_SIZE_TRAIN, 
                    validation_data = val_generator, callbacks=[monitor],
                    verbose = 1, validation_steps=STEP_SIZE_VALID)

2022-04-18 20:05:27.358662: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/200


2022-04-18 20:05:39.692901: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


450/450 [==============================] - 260s 537ms/step - loss: 99738208.0000 - rmse: 9986.9014 - val_loss: 348938208.0000 - val_rmse: 18679.8887
Epoch 2/200
450/450 [==============================] - 163s 362ms/step - loss: 41329452.0000 - rmse: 6428.7988 - val_loss: 39047700.0000 - val_rmse: 6248.8159
Epoch 3/200
450/450 [==============================] - 161s 357ms/step - loss: 20196734.0000 - rmse: 4494.0776 - val_loss: 31975226.0000 - val_rmse: 5654.6641
Epoch 4/200
450/450 [==============================] - 162s 360ms/step - loss: 16489761.0000 - rmse: 4060.7588 - val_loss: 18151574.0000 - val_rmse: 4260.4663
Epoch 5/200
450/450 [==============================] - 163s 361ms/step - loss: 15989929.0000 - rmse: 3998.7410 - val_loss: 244903360.0000 - val_rmse: 15649.3887
Epoch 6/200
450/450 [==============================] - 165s 365ms/step - loss: 15139113.0000 - rmse: 3890.9014 - val_loss: 75150656.0000 - val_rmse: 8668.9482
Epoch 7/200
450/450 [==============================] -

In [8]:
test_generator.reset()
pred = model.predict(test_generator,steps=len(df_test))

df_submit = pd.DataFrame({'id':df_test['id'],'distance':pred.flatten()})
df_submit.to_csv("submit.csv",index=False)